In [38]:
import os
import json
import pandas as pd
import pickle
import numpy as np

import re
import itertools
from collections import Counter

import tensorflow as tf
import csv

In [2]:
TPS_DIR = ''
TP_file = os.path.join(TPS_DIR, 'reviews_Musical_Instruments_5.json')

In [5]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id, 'ratings']].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [17]:
f= open(TP_file)
users_id=[]
items_id=[]
ratings=[]
reviews=[]
np.random.seed(2017)

for line in f:
    js=json.loads(line)
    if str(js['reviewerID'])=='unknown':
        print("unknown")
        continue
    if str(js['asin'])=='unknown':
        print("unknown2")
        continue
    reviews.append(js['reviewText'])
    users_id.append(str(js['reviewerID'])+',')
    items_id.append(str(js['asin'])+',')
    ratings.append(str(js['overall']))

data=pd.DataFrame({'user_id':pd.Series(users_id), 'item_id':pd.Series(items_id), 'ratings':pd.Series(ratings), 'reviews':pd.Series(reviews)})[['user_id','item_id','ratings','reviews']]
data

,user_id,item_id,ratings,reviews
0,"A2IBPI20UZIR0U,","1384719342,",5.0,"Not much to write about here, but it does exac..."
1,"A14VAT5EAX3D9S,","1384719342,",5.0,The product does exactly as it should and is q...
2,"A195EZSQDW3E21,","1384719342,",5.0,The primary job of this device is to block the...
3,"A2C00NNG1ZQQG2,","1384719342,",5.0,Nice windscreen protects my MXL mic and preven...
4,"A94QU4C90B1AX,","1384719342,",5.0,This pop filter is great. It looks and perform...
...,...,...,...,...
10256,"A14B2YH83ZXMPP,","B00JBIVXGC,",5.0,"Great, just as expected. Thank to all."
10257,"A1RPTVW5VEOSI,","B00JBIVXGC,",5.0,I've been thinking about trying the Nanoweb st...
10258,"AWCJ12KBO5VII,","B00JBIVXGC,",4.0,I have tried coated strings in the past ( incl...
10259,"A2Z7S8B5U4PAKJ,","B00JBIVXGC,",4.0,"Well, MADE by Elixir and DEVELOPED with Taylor..."


In [22]:
usercount, itemcount = get_count(data, 'user_id'), get_count(data, 'item_id')
print(f'user count: {usercount}')
print(f'item count: {itemcount}')
unique_uid = usercount.index
unique_sid = itemcount.index
print(f'unique UId: {unique_uid}')
print(f'unique SId: {unique_sid}')
item2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
user2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))
print(f'item2id: {item2id}')
print(f'user2id: {user2id}')

user count:                      user_id  size
0     A00625243BI8W1SSZNLMD,     8
1            A10044ECXDUVKS,     6
2            A102MU6ZC9H1N6,     6
3            A109JTUZXO61UY,     5
4            A109ME7C09HM2M,     5
...                      ...   ...
1424          AZJPNK73JF3XP,    11
1425          AZMHABTPXVLG3,     5
1426          AZMIKIG4BB6BZ,     5
1427          AZPDO6FLSMLFP,     7
1428          AZVME8JMPD3F4,     8

[1429 rows x 2 columns]
item count:          item_id  size
0    1384719342,     5
1    B00004Y2UT,     6
2    B00005ML71,     5
3    B000068NSX,     7
4    B000068NTU,     5
..           ...   ...
895  B00H02C9TG,     6
896  B00HFRXACG,    11
897  B00IZCSW3M,    24
898  B00J4TBMVO,     8
899  B00JBIVXGC,    13

[900 rows x 2 columns]
unique UId: RangeIndex(start=0, stop=1429, step=1)
unique SId: RangeIndex(start=0, stop=900, step=1)
item2id: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 

In [13]:
def numerize(tp):
    uid = map(lambda x: user2id[x], tp['user_id'])
    sid = map(lambda x: item2id[x], tp['item_id'])
    print(uid)
    print(sid)
    tp['user_id'] = uid
    tp['item_id'] = sid
    return tp

In [37]:
# data = numerize(data)
tp_rating = data[['user_id','item_id','ratings']]


n_ratings = tp_rating.shape[0]
test = np.random.choice(n_ratings, size=int(0.20 * n_ratings), replace=False)
test_idx = np.zeros(n_ratings, dtype=bool)
test_idx[test] = True

tp_1 = tp_rating[test_idx]
tp_train= tp_rating[~test_idx]

data2=data[test_idx]
data=data[~test_idx]


n_ratings = tp_1.shape[0]
test = np.random.choice(n_ratings, size=int(0.50 * n_ratings), replace=False)

test_idx = np.zeros(n_ratings, dtype=bool)
test_idx[test] = True

tp_test = tp_1[test_idx]
tp_valid = tp_1[~test_idx]
tp_train.to_csv(os.path.join(TPS_DIR, 'music_train.csv'), index=False,header=None)
tp_valid.to_csv(os.path.join(TPS_DIR, 'music_valid.csv'), index=False,header=None)
tp_test.to_csv(os.path.join(TPS_DIR, 'music_test.csv'), index=False,header=None)

user_reviews={}
item_reviews={}
user_rid={}
item_rid={}
for i in data.values:
    if i[0] in user_reviews.keys():
        user_reviews[i[0]].append(i[3])
        user_rid[i[0]].append(i[1])
    else:
        user_rid[i[0]]=[i[1]]
        user_reviews[i[0]]=[i[3]]
    if i[1] in item_reviews.keys():
        item_reviews[i[1]].append(i[3])
        item_rid[i[1]].append(i[0])
    else:
        item_reviews[i[1]] = [i[3]]
        item_rid[i[1]]=[i[0]]


for i in data2.values:
    if i[0] in user_reviews.keys():
        l=1
    else:
        user_rid[i[0]]=[0]
        user_reviews[i[0]]=['0']
    if i[1]in item_reviews.keys():
        l=1
    else:
        item_reviews[i[1]] = [0]
        item_rid[i[1]]=['0']

print(item_reviews)
pickle.dump(user_reviews, open(os.path.join(TPS_DIR, 'user_review'), 'wb'))
pickle.dump(item_reviews, open(os.path.join(TPS_DIR, 'item_review'), 'wb'))
pickle.dump(user_rid, open(os.path.join(TPS_DIR, 'user_rid'), 'wb'))
pickle.dump(item_rid, open(os.path.join(TPS_DIR, 'item_rid'), 'wb'))

usercount, itemcount = get_count(data, 'user_id'), get_count(data, 'item_id')

print(f'usercount: {usercount}')
print(f'itemcount: {itemcount}')

print(usercount.sort_values(by='size'))

print(itemcount.sort_values(by='size'))

{'1384719342,': ["This pop filter is great. It looks and performs like a studio filter. If you're recording vocals this will eliminate the pops that gets recorded when you sing."], 'B000068NSX,': ['Very good cable. Well made and it looks great with my Candy Apple Red Tele. The reason I gave it only 4 stars is because it tends not to lay flat on the floor.'], 'B000068NW5,': ["I'm not a professional, but have been playing for many, many years.  This is the best cord I've ever had, though I'm sure Eric Clapton has something better."], 'B000068O1N,': ["When you need an audio insert cable, you need an audio insert cable.We custom manufacture insert cables for custom sound-reinforcement fixed installations.  For  mobile rigs, such as mixer cases with processing gear, it's simply not cost effective, and Hosa is our go-to brand for reliability and durability.  There are other brands, but our reputation is at stake.Hosa allows us to maintain our good reputation, while lowering our labor costs. 

pro_data.py

In [39]:
tf.flags.DEFINE_string("valid_data","../data/music/music_valid.csv", " Data for validation")
tf.flags.DEFINE_string("test_data", "../data/music/music_test.csv", "Data for testing")
tf.flags.DEFINE_string("train_data", "../data/music/music_train.csv", "Data for training")
tf.flags.DEFINE_string("user_review", "../data/music/user_review", "User's reviews")
tf.flags.DEFINE_string("item_review", "../data/music/item_review", "Item's reviews")

AttributeError: module 'tensorflow' has no attribute 'flags'

In [41]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def pad_sentences(u_text,u_len,padding_word="<PAD/>"):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
    sequence_length=u_len
    u_text2={}
    print(len(u_text))
    for i in u_text.keys():
        #print i
        sentence = u_text[i]
        if sequence_length>len(sentence):

            num_padding = sequence_length - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
            u_text2[i]=new_sentence
        else:
            new_sentence = sentence[:sequence_length]
            u_text2[i] = new_sentence

    return u_text2
def build_vocab(sentences1,sentences2):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts1 = Counter(itertools.chain(*sentences1))
    # Mapping from index to word
    vocabulary_inv1 = [x[0] for x in word_counts1.most_common()]
    vocabulary_inv1 = list(sorted(vocabulary_inv1))
    # Mapping from word to index
    vocabulary1 = {x: i for i, x in enumerate(vocabulary_inv1)}

    word_counts2 = Counter(itertools.chain(*sentences2))
    # Mapping from index to word
    vocabulary_inv2 = [x[0] for x in word_counts2.most_common()]
    vocabulary_inv2 = list(sorted(vocabulary_inv2))
    # Mapping from word to index
    vocabulary2 = {x: i for i, x in enumerate(vocabulary_inv2)}
    return [vocabulary1, vocabulary_inv1,vocabulary2, vocabulary_inv2]

def build_input_data(u_text,i_text, vocabulary_u,vocabulary_i):
    """
    Maps sentencs and labels to vectors based on a vocabulary.
    """
    l = len(u_text)
    u_text2 = {}
    for i in u_text.keys():
        u_reviews = u_text[i]
        u = np.array([vocabulary_u[word] for word  in u_reviews])
        u_text2[i] = u
    l = len(i_text)
    i_text2 = {}
    for j in i_text.keys():
        i_reviews = i_text[j]
        i = np.array([vocabulary_i[word] for word in i_reviews])
        i_text2[j] = i
    return u_text2, i_text2


def load_data(train_data,valid_data,user_review,item_review):
    """
    Loads and preprocessed data for the MR dataset.
    Returns input vectors, labels, vocabulary, and inverse vocabulary.
    """
    # Load and preprocess data
    u_text,i_text, y_train, y_valid,u_len,i_len,uid_train,iid_train,uid_valid,iid_valid,user_num,item_num=\
        load_data_and_labels(train_data,valid_data,user_review,item_review)
    print("load data done")
    u_text = pad_sentences(u_text,u_len)
    print("pad user done")
    i_text=pad_sentences(i_text,i_len)
    print("pad item done")

    user_voc = [x for x in u_text.itervalues()]
    item_voc = [x for x in i_text.itervalues()]

    vocabulary_user, vocabulary_inv_user, vocabulary_item, vocabulary_inv_item = build_vocab(user_voc, item_voc)
    print(len(vocabulary_user))
    print(len(vocabulary_item))
    u_text, i_text = build_input_data(u_text, i_text, vocabulary_user, vocabulary_item)
    y_train = np.array(y_train)
    y_valid = np.array(y_valid)
    uid_train = np.array(uid_train)
    uid_valid = np.array(uid_valid)
    iid_train = np.array(iid_train)
    iid_valid = np.array(iid_valid)

    return [u_text,i_text,y_train,y_valid, vocabulary_user, vocabulary_inv_user,vocabulary_item,vocabulary_inv_item,
            uid_train, iid_train, uid_valid, iid_valid, user_num, item_num]


def load_data_and_labels(train_data,valid_data,user_review,item_review):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    f_train = open(train_data, "r")
    f1=open(user_review)
    f2=open(item_review)

    user_reviews=pickle.load(f1)
    item_reviews=pickle.load(f2)
    uid_train = []
    iid_train = []
    y_train=[]
    u_text = {}
    i_text = {}
    i = 0
    for line in f_train:
        i = i + 1
        line = line.split(',')
        uid_train.append(int(line[0]))
        iid_train.append(int(line[1]))
        if u_text.has_key(int(line[0])):
            a=1
        else:
            u_text[int(line[0])] = '<PAD/>'
            for s in user_reviews[int(line[0])]:
                u_text[int(line[0])] = u_text[int(line[0])] + ' ' + s.strip()
            u_text[int(line[0])]=clean_str(u_text[int(line[0])])
            u_text[int(line[0])]=u_text[int(line[0])].split(" ")

        if i_text.has_key(int(line[1])):
            a=1
        else:
            i_text[int(line[1])] = '<PAD/>'
            for s in item_reviews[int(line[1])]:
                i_text[int(line[1])] = i_text[int(line[1])] + ' ' + s.strip()
            i_text[int(line[1])]=clean_str(i_text[int(line[1])])
            i_text[int(line[1])]=i_text[int(line[1])].split(" ")
        y_train.append(float(line[2]))
    print("valid")

    uid_valid = []
    iid_valid = []
    y_valid=[]
    f_valid=open(valid_data)
    for line in f_valid:
        line=line.split(',')
        uid_valid.append(int(line[0]))
        iid_valid.append(int(line[1]))
        if u_text.has_key(int(line[0])):
            a=1
        else:
            u_text[int(line[0])] = '<PAD/>'
            u_text[int(line[0])]=clean_str(u_text[int(line[0])])
            u_text[int(line[0])]=u_text[int(line[0])].split(" ")


        if i_text.has_key(int(line[1])):
            a=1
        else:
            i_text[int(line[1])] = '<PAD/>'
            i_text[int(line[1])]=clean_str(i_text[int(line[1])])
            i_text[int(line[1])]=i_text[int(line[1])].split(" ")

        y_valid.append(float(line[2]))
    print("len")
    u = np.array([len(x) for x in u_text.itervalues()])
    x = np.sort(u)
    u_len = x[int(0.85* len(u)) - 1]


    i = np.array([len(x) for x in i_text.itervalues()])
    y = np.sort(i)
    i_len = y[int(0.85 * len(i)) - 1]
    print("u_len:",u_len)
    print("i_len:",i_len)
    user_num = len(u_text)
    item_num = len(i_text)
    print("user_num:", user_num)
    print("item_num:", item_num)
    return [u_text,i_text,y_train,y_valid,u_len,i_len,uid_train,iid_train,uid_valid,iid_valid,user_num,item_num]


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [47]:
if __name__ == '__main__':
    TPS_DIR = '../data/music'
    f = tf.compat.v1.app.flags
    f.FLAGS.parse()

    u_text,i_text, y_train, y_valid, vocabulary_user, vocabulary_inv_user, vocabulary_item, \
    vocabulary_inv_item, uid_train, iid_train, uid_valid, iid_valid, user_num, item_num = \
        load_data(FLAGS.train_data, FLAGS.valid_data, FLAGS.user_review, FLAGS.item_review)

    np.random.seed(2017)

    shuffle_indices = np.random.permutation(np.arange(len(y_train)))

    userid_train = uid_train[shuffle_indices]
    itemid_train = iid_train[shuffle_indices]
    y_train = y_train[shuffle_indices]
    y_train = y_train[:, np.newaxis]
    y_valid = y_valid[:, np.newaxis]

    userid_train = userid_train[:, np.newaxis]
    itemid_train = itemid_train[:, np.newaxis]
    userid_valid = uid_valid[:, np.newaxis]
    itemid_valid = iid_valid[:, np.newaxis]

    batches_train=list(zip( userid_train, itemid_train, y_train))
    batches_test=list(zip(userid_valid,itemid_valid,y_valid))
    output = open(os.path.join(TPS_DIR, 'music.train'), 'wb')
    pickle.dump(batches_train,output)
    output = open(os.path.join(TPS_DIR, 'music.test'), 'wb')
    pickle.dump(batches_test,output)

    para={}
    para['user_num']=user_num
    para['item_num']=item_num
    para['user_length']=u_text[0].shape[0]
    para['item_length'] = i_text[0].shape[0]
    para['user_vocab'] = vocabulary_user
    para['item_vocab'] = vocabulary_item
    para['train_length']=len(y_train)
    para['test_length']=len(y_valid)
    para['u_text'] = u_text
    para['i_text'] = i_text
    output = open(os.path.join(TPS_DIR, 'music.para'), 'wb')

    pickle.dump(para, output)

UnrecognizedFlagError: Unknown command line flag 'f'